In [34]:
from pymongo import MongoClient
import pandas as pd
import pprint
import pymongo
import requests
import json


# import api_key
from api_key import sports_key


In [35]:
# API details
api_key = sports_key
# List of NFL team codes
teams = [
    'ARI',  # Arizona Cardinals
    'ATL',  # Atlanta Falcons
    'BAL',  # Baltimore Ravens
    'BUF',  # Buffalo Bills
    'CAR',  # Carolina Panthers
    'CHI',  # Chicago Bears
    'CIN',  # Cincinnati Bengals
    'CLE',  # Cleveland Browns
    'DAL',  # Dallas Cowboys
    'DEN',  # Denver Broncos
    'DET',  # Detroit Lions
    'GB',   # Green Bay Packers
    'HOU',  # Houston Texans
    'IND',  # Indianapolis Colts
    'JAX',  # Jacksonville Jaguars
    'KC',   # Kansas City Chiefs
    'LAC',  # Los Angeles Chargers
    'LAR',  # Los Angeles Rams
    'LV',   # Las Vegas Raiders
    'MIA',  # Miami Dolphins
    'MIN',  # Minnesota Vikings
    'NE',   # New England Patriots
    'NO',   # New Orleans Saints
    'NYG',  # New York Giants
    'NYJ',  # New York Jets
    'PHI',  # Philadelphia Eagles
    'PIT',  # Pittsburgh Steelers
    'SF',   # San Francisco 49ers
    'SEA',  # Seattle Seahawks
    'TB',   # Tampa Bay Buccaneers
    'TEN',  # Tennessee Titans
    'WAS'   # Washington Commanders
]  # Add all teams

# Initialize a dictionary to store each team's roster DataFrame
team_rosters = {}

# Loop through each team code
for team in teams:
    # Construct the URL for each team
    url = f'https://api.sportsdata.io/v3/nfl/scores/json/PlayersBasic/{team}?key={api_key}'
    
    # Make the API call
    response = requests.get(url)
    
    # Check if the response is valid
    if response.status_code == 200:
        data = response.json()
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Add the team's roster to the dictionary
        team_rosters[team] = df
    else:
        print(f"Failed to retrieve data for {team}, status code: {response.status_code}")

# Access individual team rosters
# Example: Display roster for the New England Patriots (NE)
print(team_rosters['LAC'].head())

   PlayerID Team  Number    FirstName   LastName Position          Status  \
0     22556  LAC     NaN        Jaret  Patterson       RB  Practice Squad   
1     22763  LAC     NaN        Shaun       Wade       CB  Practice Squad   
2     23831  LAC     NaN  Christopher     Hinton       DL  Practice Squad   
3     24830  LAC     NaN           CJ      Okoye       DL  Practice Squad   
4     24915  LAC     NaN     Nehemiah    Shelton       CB  Practice Squad   

  Height  Weight            BirthDate  ...    ShortName  HeightFeet  \
0   5'8"     195  1999-12-23T00:00:00  ...  J.Patterson           5   
1   6'1"     195  1998-09-16T00:00:00  ...       S.Wade           6   
2   6'4"     310  2000-09-14T00:00:00  ...     C.Hinton           6   
3   6'6"     315  2001-10-20T00:00:00  ...      C.Okoye           6   
4   6'2"     185  1999-09-13T00:00:00  ...    N.Shelton           6   

  HeightInches  TeamID GlobalTeamID UsaTodayPlayerID  \
0            8      29           29          8314692  

In [36]:
for team, roster in team_rosters.items():
    roster.to_csv(f'Resources/{team}_roster.csv', index=False)

In [37]:
# API key and season variable
season = 2023  # Replace with the desired season

# Construct the API URL
url = f'https://api.sportsdata.io/v3/nfl/stats/json/PlayerSeasonStats/{season}?key={api_key}'

# Make the API call
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON
    data = response.json()

    # Convert the data to a DataFrame
    stats_df = pd.DataFrame(data)

    # Display the first few rows of the DataFrame
    print(stats_df.head())
else:
    print(f"Failed to retrieve data: {response.status_code}")

   PlayerID  SeasonType  Season Team  Number       Name Position  \
0       549           1    2023  ARI       5   M.Prater        K   
1       611           1    2023  CLE      15   J.Flacco       QB   
2      1694           1    2023  NYG      21   M.Crosby        K   
3      2593           1    2023  NYJ       8  A.Rodgers       QB   
4      3061           1    2023  CHI      84    M.Lewis       TE   

  PositionCategory  Activated  Played  ...  AverageDraftPosition  \
0               ST         17      17  ...                  None   
1              OFF          5       5  ...                  None   
2               ST          3       3  ...                  None   
3              OFF          1       1  ...                  None   
4              OFF         17      17  ...                  None   

   AverageDraftPositionPPR  TeamID  GlobalTeamID  FantasyPointsFantasyDraft  \
0                     None       1             1                        0.0   
1                     No

In [39]:
# Loop through each team to update their rosters with player stats
for team, df in team_rosters.items():
    try:
        # Merge team roster with player stats based on PlayerID or another common field
        updated_roster_df = pd.merge(df, stats_df, on='PlayerID', how='left')
        
        # Fill NaN values (for players not found in the API) with 0 for their stats
        updated_roster_df.fillna(0, inplace=True)

        # Replace the team's roster with the updated one containing player stats
        team_rosters[team] = updated_roster_df

        # Optionally, save the updated roster to a CSV file
        updated_roster_df.to_csv(f'Resources/{team}_roster.csv', index=False)

        print(f"{team} roster updated with player stats, missing player stats set to 0.")
    except Exception as e:
        print(f"Error updating {team} roster: {e}")

ARI roster updated with player stats, missing player stats set to 0.
ATL roster updated with player stats, missing player stats set to 0.
BAL roster updated with player stats, missing player stats set to 0.
BUF roster updated with player stats, missing player stats set to 0.
CAR roster updated with player stats, missing player stats set to 0.
CHI roster updated with player stats, missing player stats set to 0.
CIN roster updated with player stats, missing player stats set to 0.
CLE roster updated with player stats, missing player stats set to 0.
DAL roster updated with player stats, missing player stats set to 0.
DEN roster updated with player stats, missing player stats set to 0.
DET roster updated with player stats, missing player stats set to 0.
GB roster updated with player stats, missing player stats set to 0.
HOU roster updated with player stats, missing player stats set to 0.
IND roster updated with player stats, missing player stats set to 0.
JAX roster updated with player stat

In [33]:
# Construct the API URL
url = f'https://api.sportsdata.io/v3/nfl/stats/json/PlayerSeasonStats/{season}?key={api_key}'

# Make the API call
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON
    data = response.json()

    # Convert the data to a DataFrame
    all_players_df = pd.DataFrame(data)

    # Display the first few rows of the DataFrame
    all_players_df.to_csv('Resources/all_players.csv', index=False)
    print("all_players.csv has been saved.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

all_players.csv has been saved.
